In [1]:
# python -m pip install -r requirements.txt

# 위의 파일이 실행이 안된다면
# python -m pip install selenium webdriver-manager beautifulsoup4 pandas


In [2]:
# 1) 라이브러리 불러오기
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
#Selenium 드라이버 설정 (headless 모드) -> BS로 안열리는 경우 이걸로 우회
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)

In [4]:
# 페이지 열기
url = 'https://www.basketball-reference.com/leagues/NBA_2024_per_game.html'
driver.get(url)

# 렌더된 HTML 가져오기
html = driver.page_source

# Selenium 드라이버 종료
driver.quit()

In [5]:
# BeautifulSoup 으로 파싱
soup = BeautifulSoup(html, 'html.parser')

# 테이블 선택
table = soup.find('table', id='per_game_stats')

# thead에서 컬럼명 추출 (첫 번째 th 제거)
headers = [th.get_text(strip=True) for th in table.thead.find_all('th')]

# tbody에서 선수별 데이터 추출
rows = []
for tr in table.tbody.find_all('tr'):
    if tr.get('class') and 'thead' in tr['class']:
        continue
    rank = tr.find('th').get_text(strip=True)
    stats = [td.get_text(strip=True) for td in tr.find_all('td')]
    rows.append([rank] + stats)


In [6]:
# DataFrame 생성 및 출력
df = pd.DataFrame(rows, columns=headers)
print(df[['Player','Pos','G','GS','MP']].head())

                    Player Pos   G  GS    MP
0              Joel Embiid   C  39  39  33.6
1              Luka Dončić  PG  70  70  37.5
2    Giannis Antetokounmpo  PF  73  73  35.2
3  Shai Gilgeous-Alexander  PG  75  75  34.0
4            Jalen Brunson  PG  77  77  35.4


In [7]:
# csv로 내보내기
df.to_csv('nba_2024_per_game.csv', index=False)